# Nate Warner: Project Proposal

#### AccurateArithmetic.jl: Kahan's summation algorithm for compensated summations and more

Provide:

    1. an explanation of why you chose this package;
    2. a simple code example showing how the package works;
    3. a description of what you plan to present.

1. Browsing online for interesting Julia packages I came across AccurateArithmetic.jl, which implements Kahan's summation algorithm. Consider a sum of reals
    $$ S_n = \sum_{i=0}^{n}a_{i}$$
   for $a_i \in \mathbb{R}$. The naive approach to implement an algorithm that computes this sum could be extremely imprecise due to things like rounding error, cancellation error, and error in $x+y$ when $x>>y$. Kahan's algorithm reduces the errors that would normally accumlate in the naive approach.

   The github page for the package states "AccurateArithmetic.jl provides a set of error-free transforms (EFTs), which allow getting not only the rounded result of a floating-point computation, but also the accompanying rounding erro. EFTs can be leveraged to build "compensated algorithms", which compute a result as if the basic algorithm had been run using a higher precision."

We note that Kahan's algorithm promises $O(1)$ error growth, where the naive approach is $O(n)$. Kahan's algorithm is implemented as _sum_kbn_() in the AccurateArithmetic package.

2.) Let's first consider an example of summing many values.

In [110]:
#Naive approach
function naivesum(x, double=false)
    sum=double ? 0.0 : Float32(0)
    for i in eachindex(x)
        sum+=x[i]
    end
    sum
end

naivesum (generic function with 2 methods)

In [186]:
x = Float32[1e16^(-i) for i in 0:0.001:100]
(naivesum(x), sum_kbn(x))

(27.646463f0, 27.646475f0)

We can check which one is more accurate by checking against the Float64 sum.

In [189]:
x = [1e16^(-i) for i in 0:0.001:100]
naivesum(x), sum_kbn(x)

(27.6464751629624, 27.646475162962446)

We see that when using less precise Float32, we can get extra digits of precision by using Kahan's algorithm.

Next, we consider an example using the input $\{1e16,1,1e-16\}$.

In [190]:
(a,b,c) = (1e16, 1,-1e16)
a+b+c

0.0

Since 1 is added to $10^{16}$, the information in 1 is lost due to the magnitude of $10^{16}$. Thus, subtracting $10^{16}$ from the result yields zero. Let's see if sum_kbn does any better.

In [223]:
x = [1e16,1,-1e16]
sum_kbn(x)

1.0

And we see that it does.

I'd like to also show the example found in the package documentation.

In [217]:
# By design, this vector sums to 1
x = 5000 |> N->randn(N) .* exp.(10 .* randn(N)) |> x->[x;-x;1.0] |> x->x[sortperm(rand(length(x)))];
sum(big.(x))

1.0

Let's what naivesum yields.

In [218]:
naivesum(x)

-11.390746881804123

We see that the result of naivesum is very far off from the expected result of one. Next, let's see how sum_kbn does.

In [222]:
sum_kbn(x)

1.0000000000000042

We see that the result is very close to the expected one.

3.) In the presentation I would like to show 

    - The naive approach for summing reals with bounded precision,
    - The numerical problems faced in these sums,
    - Kahan's algorithm and how it overcomes these problems,
    - AccurateArithmetic.jl's interface for Kahan's algorithm,
    - Other useful functions defined in the package,
    - Errors.

We note that AccurateArithmetic has several useful functions besides sum_kbn. If there is time I will show those as well.